In [0]:
import pandas as pd
import time
import numpy as np
import matplotlib.pyplot as plt
import re
%matplotlib inline

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
path = "/content/drive/Shared drives/MotionAI/"

### Load and Wrangle Data

In [0]:
df = pd.concat([pd.read_csv(path+"walking.csv"), pd.read_csv(path+"falling.csv"), pd.read_csv(path+"sitting.csv"), pd.read_csv(path+"lying.csv")])

In [5]:
df.columns = ['Image Name', 'Chute Num', 'Cam Num', 'Frame Num', 'Points', 'Label']
df.set_index('Image Name', inplace = True)
df.head()

,Chute Num,Cam Num,Frame Num,Points,Label
Image Name,,,,,
03_cam7.avi_frame460.jpg,3,cam7.avi,frame460.jpg,"[(390, 54), (390, 68), (390, 68), None, None, ...",walking
03_cam7.avi_frame420.jpg,3,cam7.avi,frame420.jpg,"[(576, 82), (596, 96), (576, 96), (576, 123), ...",walking
03_cam7.avi_frame425.jpg,3,cam7.avi,frame425.jpg,"[(555, 82), (576, 96), (555, 96), (555, 123), ...",walking
03_cam7.avi_frame465.jpg,3,cam7.avi,frame465.jpg,"[(349, 54), (370, 68), (370, 68), None, None, ...",walking
03_cam7.avi_frame430.jpg,3,cam7.avi,frame430.jpg,"[(534, 68), (534, 96), (534, 96), (514, 109), ...",walking


In [0]:
# Create list of 36 co-ordinates
df['Points'] = df['Points'].apply(lambda x: x.replace('None', '(None, None)'))
df['Points'] = df['Points'].apply(lambda x: re.findall(r'\d+|None', x))

In [7]:
df.head()

,Chute Num,Cam Num,Frame Num,Points,Label
Image Name,,,,,
03_cam7.avi_frame460.jpg,3,cam7.avi,frame460.jpg,"[390, 54, 390, 68, 390, 68, None, None, None, ...",walking
03_cam7.avi_frame420.jpg,3,cam7.avi,frame420.jpg,"[576, 82, 596, 96, 576, 96, 576, 123, None, No...",walking
03_cam7.avi_frame425.jpg,3,cam7.avi,frame425.jpg,"[555, 82, 576, 96, 555, 96, 555, 123, None, No...",walking
03_cam7.avi_frame465.jpg,3,cam7.avi,frame465.jpg,"[349, 54, 370, 68, 370, 68, None, None, None, ...",walking
03_cam7.avi_frame430.jpg,3,cam7.avi,frame430.jpg,"[534, 68, 534, 96, 534, 96, 514, 109, 514, 123...",walking


In [0]:
point_cols = ['Nose_x',
 'Nose_y',
 'Neck_x',
 'Neck_y',
 'Right Shoulder_x',
 'Right Shoulder_y',
 'Right Elbow_x',
 'Right Elbow_y',
 'Right Wrist_x',
 'Right Wrist_y',
 'Left Shoulder_x',
 'Left Shoulder_y',
 'Left Elbow_x',
 'Left Elbow_y',
 'Left Wrist_x',
 'Left Wrist_y',
 'Right Hip_x',
 'Right Hip_y',
 'Right Knee_x',
 'Right Knee_y',
 'Right Ankle_x',
 'Right Ankle_y',
 'Left Hip_x',
 'Left Hip_y',
 'Left Knee_x',
 'Left Knee_y',
 'LAnkle_x',
 'LAnkle_y',
 'Right Eye_x',
 'Right Eye_y',
 'Left Eye_x',
 'Left Eye_y',
 'Right Ear_x',
 'Right Ear_y',
 'Left Ear_x',
 'Left Ear_y']

In [0]:
df[point_cols] = pd.DataFrame(df.Points.values.tolist(), index = df.index) # Split list into columns
df.drop(columns = ['Points'], inplace = True ) 
#df = df.replace("None", pd.np.nan).dropna(axis = 0, how = 'all', subset = df.columns[4:]) # Drop rows where all null
df = df.replace("None", pd.np.nan)
df.iloc[:,4:] = df.iloc[:,4:].astype(float) # Conver to float

In [0]:
df['Frame Num'] = df["Frame Num"].apply(lambda x: int(re.findall('\\d+', x)[0]))
df.sort_values(by = ['Chute Num', 'Cam Num', 'Frame Num'], inplace = True)

In [0]:
df.loc['03_cam2.avi_frame625.jpg'] = df.loc['03_cam2.avi_frame630.jpg']

In [0]:
# Scaling
x_cols = [col for col in df.columns if col.count('_x') == 1]
y_cols = [col for col in df.columns if col.count('_y') == 1]

df[x_cols] = df[x_cols]/720
df[y_cols] = df[y_cols]/480

In [0]:
df = df.replace(pd.np.nan, -1)

In [54]:
df.head(20)

,Chute Num,Cam Num,Frame Num,Label,Nose_x,Nose_y,Neck_x,Neck_y,Right Shoulder_x,Right Shoulder_y,Right Elbow_x,Right Elbow_y,Right Wrist_x,Right Wrist_y,Left Shoulder_x,Left Shoulder_y,Left Elbow_x,Left Elbow_y,Left Wrist_x,Left Wrist_y,Right Hip_x,Right Hip_y,Right Knee_x,Right Knee_y,Right Ankle_x,Right Ankle_y,Left Hip_x,Left Hip_y,Left Knee_x,Left Knee_y,LAnkle_x,LAnkle_y,Right Eye_x,Right Eye_y,Left Eye_x,Left Eye_y,Right Ear_x,Right Ear_y,Left Ear_x,Left Ear_y
Image Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
01_cam1.avi_frame865.jpg,1,cam1.avi,865,walking,-1.000000,-1.000000,0.113889,0.056250,0.113889,0.056250,0.113889,0.085417,0.113889,0.112500,0.113889,0.056250,-1.000000,-1.000000,-1.000000,-1.000000,0.113889,0.112500,0.113889,0.170833,0.113889,0.227083,0.113889,0.112500,0.113889,0.170833,0.113889,0.227083,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.00000
01_cam1.avi_frame870.jpg,1,cam1.avi,870,walking,0.141667,0.027083,0.141667,0.056250,0.113889,0.056250,0.113889,0.085417,0.113889,0.141667,0.141667,0.056250,-1.000000,-1.000000,-1.000000,-1.000000,0.113889,0.112500,0.113889,0.170833,0.113889,0.227083,0.141667,0.112500,0.141667,0.170833,0.141667,0.227083,0.141667,0.027083,-1.000000,-1.000000,0.141667,0.027083,-1.000000,-1.00000
01_cam1.avi_frame875.jpg,1,cam1.avi,875,walking,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.00000
01_cam1.avi_frame880.jpg,1,cam1.avi,880,walking,0.170833,0.027083,0.170833,0.056250,0.170833,0.056250,0.170833,0.085417,0.170833,0.112500,0.170833,0.056250,0.170833,0.085417,0.200000,0.112500,0.170833,0.112500,0.170833,0.170833,0.141667,0.227083,0.170833,0.112500,0.170833,0.170833,0.170833,0.227083,0.170833,0.027083,0.170833,0.027083,0.170833,0.027083,-1.000000,-1.00000
01_cam1.avi_frame885.jpg,1,cam1.avi,885,walking,0.200000,0.027083,0.170833,0.056250,0.170833,0.056250,0.170833,0.085417,0.227778,0.112500,0.200000,0.056250,0.200000,0.085417,0.227778,0.112500,0.170833,0.141667,0.170833,0.200000,0.170833,0.256250,0.200000,0.141667,0.170833,0.200000,0.170833,0.227083,0.200000,0.027083,0.200000,0.027083,0.200000,0.027083,-1.000000,-1.00000
01_cam1.avi_frame890.jpg,1,cam1.avi,890,walking,-1.000000,-1.000000,0.200000,0.056250,0.200000,0.056250,0.227778,0.112500,0.227778,0.112500,0.200000,0.056250,0.170833,0.112500,0.170833,0.141667,0.200000,0.141667,0.200000,0.200000,0.200000,0.256250,0.200000,0.141667,0.200000,0.200000,0.170833,0.227083,-1.000000,-1.000000,-1.000000,-1.000000,0.200000,0.056250,-1.000000,-1.00000
01_cam1.avi_frame895.jpg,1,cam1.avi,895,walking,0.227778,0.056250,0.227778,0.056250,0.227778,0.056250,0.227778,0.112500,0.227778,0.141667,0.227778,0.056250,-1.000000,-1.000000,-1.000000,-1.000000,0.227778,0.141667,0.227778,0.200000,-1.000000,-1.000000,0.227778,0.141667,0.227778,0.200000,-1.000000,-1.000000,0.227778,0.056250,-1.000000,-1.000000,0.227778,0.056250,-1.000000,-1.00000
01_cam1.avi_frame900.jpg,1,cam1.avi,900,walking,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.00000
01_cam1.avi_frame905.jpg,1,cam1.avi,905,walking,0.284722,0.056250,0.284722,0.085417,0.256944,0.085417,0.256944,0.112500,0.256944,0.141667,0.284722,0.085417,0.284722,0.112500,-1.000000,-1.000000,0.256944,0.141667,0.284722,0.200000,0.284722,0.256250,0.284722,0.141667,0.256944,0.200000,0.227778,0.256250,0.284722,0.056250,0.284722,0.056250,0.284722,0.056250,-1.000000,-1.00000


In [57]:
df.shape

(5760, 40)

In [0]:
df.to_csv(path+"batch1.csv")